In [1]:
import numpy as np

data = np.load('Mg22_size512_convertXYZQ_train.npy')


In [6]:
import random

num_chunks = 8  # or adjust based on your data's dimensions
points_per_chunk = 32  # or adjust as needed
z_dim = data.shape[0]  # z-dimension of your data

print(z_dim)
start_points = random.sample(range(z_dim - points_per_chunk + 1), num_chunks)


2015


In [ ]:
def remove_points_in_segments(event, num_segments, total_points_to_remove):
    # Determine the number of points to remove per segment
    points_per_segment = total_points_to_remove // num_segments

    # Get the range of Z values
    z_min, z_max = np.min(event[:, 2]), np.max(event[:, 2])
    segment_size = (z_max - z_min) / num_segments

    modified_event = event.copy()

    for i in range(num_segments):
        # Define the segment bounds
        z_lower = z_min + i * segment_size
        z_upper = z_lower + segment_size

        # Select points within this segment
        segment_mask = (modified_event[:, 2] >= z_lower) & (modified_event[:, 2] < z_upper)
        segment_points = modified_event[segment_mask]

        # Randomly remove points from this segment
        if len(segment_points) > points_per_segment:
            points_to_remove = np.random.choice(range(len(segment_points)), points_per_segment, replace=False)
            segment_points = np.delete(segment_points, points_to_remove, axis=0)

        # Update the modified event
        modified_event = np.concatenate([modified_event[~segment_mask], segment_points])

    return modified_event

# Apply the new function to remove points
modified_event_v2 = remove_points_in_segments(first_event, 8, 256)

# 3D Plotting the new modified event
x_mod_v2 = modified_event_v2[:, 0]
y_mod_v2 = modified_event_v2[:, 1]
z_mod_v2 = modified_event_v2[:, 2]

fig_v2 = plt.figure(figsize=(10, 8))
ax_v2 = fig_v2.add_subplot(111, projection='3d')
scatter_v2 = ax_v2.scatter(x_mod_v2, y_mod_v2, z_mod_v2, c=z_mod_v2, cmap='viridis')
plt.colorbar(scatter_v2, ax=ax_v2, label='Z Coordinate')
ax_v2.set_xlabel('X Coordinate')
ax_v2.set_ylabel('Y Coordinate')
ax_v2.set_zlabel('Z Coordinate')
ax_v2.set_title('3D Scatter Plot of New Modified Event')
plt.show()
